In [12]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
import json

In [15]:
from sklearn.datasets import load_iris

In [9]:
iris = datasets.load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)

In [11]:
df.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)'],
      dtype='object')

In [23]:
iris = load_iris()
feature_name = ['SL', 'SW', 'PL', 'PW']
df = pd.DataFrame(iris.data, columns=feature_name)
json_records = df.to_json(orient ='records')
data = json.loads(json_records)
context = {'d': data}
context

{'d': [{'SL': 5.1, 'SW': 3.5, 'PL': 1.4, 'PW': 0.2},
  {'SL': 4.9, 'SW': 3.0, 'PL': 1.4, 'PW': 0.2},
  {'SL': 4.7, 'SW': 3.2, 'PL': 1.3, 'PW': 0.2},
  {'SL': 4.6, 'SW': 3.1, 'PL': 1.5, 'PW': 0.2},
  {'SL': 5.0, 'SW': 3.6, 'PL': 1.4, 'PW': 0.2},
  {'SL': 5.4, 'SW': 3.9, 'PL': 1.7, 'PW': 0.4},
  {'SL': 4.6, 'SW': 3.4, 'PL': 1.4, 'PW': 0.3},
  {'SL': 5.0, 'SW': 3.4, 'PL': 1.5, 'PW': 0.2},
  {'SL': 4.4, 'SW': 2.9, 'PL': 1.4, 'PW': 0.2},
  {'SL': 4.9, 'SW': 3.1, 'PL': 1.5, 'PW': 0.1},
  {'SL': 5.4, 'SW': 3.7, 'PL': 1.5, 'PW': 0.2},
  {'SL': 4.8, 'SW': 3.4, 'PL': 1.6, 'PW': 0.2},
  {'SL': 4.8, 'SW': 3.0, 'PL': 1.4, 'PW': 0.1},
  {'SL': 4.3, 'SW': 3.0, 'PL': 1.1, 'PW': 0.1},
  {'SL': 5.8, 'SW': 4.0, 'PL': 1.2, 'PW': 0.2},
  {'SL': 5.7, 'SW': 4.4, 'PL': 1.5, 'PW': 0.4},
  {'SL': 5.4, 'SW': 3.9, 'PL': 1.3, 'PW': 0.4},
  {'SL': 5.1, 'SW': 3.5, 'PL': 1.4, 'PW': 0.3},
  {'SL': 5.7, 'SW': 3.8, 'PL': 1.7, 'PW': 0.3},
  {'SL': 5.1, 'SW': 3.8, 'PL': 1.5, 'PW': 0.3},
  {'SL': 5.4, 'SW': 3.4, 'PL': 1.7,

In [2]:
def main(code):
    res = requests.get(BASE_URL + str(code) + '&page=' + str(START_PAGE))
    page_soup = BeautifulSoup(res.text, 'lxml')
    
    total_page_num = page_soup.select_one('td.pgRR > a')
    total_page_num = int(total_page_num.get('href').split('=')[-1])
    
    ipt_html = page_soup.select_one('div.subcnt_sise_item_top')
    global fields
    fields = [item.get('value') for item in ipt_html.select('input')]
    
    result = [crawl(code,str(page)) for page in range(1, total_page_num + 1)]
    
    df = pd.concat(result, axis = 0, ignore_index = True)
    return df
    
def crawl(code, page):
    global fields
    data = {'menu': 'market_sum',
            'fieldIds': fields,
            'returnUrl': BASE_URL + str(code) + "&page=" + str(page)}
    
    res = requests.post('https://finance.naver.com/sise/field_submit.nhn', data = data)
    page_soup = BeautifulSoup(res.text, 'lxml')
    
    table_html = page_soup.select_one('div.box_type_l')
    
    header_data = [item.get_text().strip() for item in table_html.select('thead th')][1:-1]
    
    inner_data = [item.get_text().strip() for item in table_html.find_all(lambda x:
                                                                            (x.name == 'a' and
                                                                             'tltle' in x.get('class', [])) or
                                                                            (x.name == 'td' and
                                                                             'number' in x.get('class', []))
                                                                            )]
    no_data = [item.get_text().strip() for item in table_html.select('td.no')]
    number_data = np.array(inner_data)
    
    number_data.resize(len(no_data), len(header_data))
    
    df = pd.DataFrame(data = number_data, columns = header_data)
    return df

BASE_URL = 'https://finance.naver.com/sise/sise_market_sum.nhn?sosok='

KOSPI_CODE = 0
KOSDAK_CODE = 1
START_PAGE = 1
field = []
KOSPI = main(KOSPI_CODE)
KOSDAK = main(KOSDAK_CODE)
KOSPI = KOSPI.replace('N/A',np.nan).dropna(axis = 0)
KOSPI['PER'] = KOSPI['PER'].str.replace(',','').astype('float64')
KOSPI['ROE'] = KOSPI['ROE'].str.replace(',','').astype('float64')
KOSPI['ROA'] = KOSPI['ROA'].str.replace(',','').astype('float64')
KOSPI['시가총액'] = KOSPI['시가총액'].str.replace(',','').astype('int64')
KOSPI['매출액'] = KOSPI['매출액'].str.replace(',','').astype('int64')
KOSPI['영업이익'] = KOSPI['영업이익'].str.replace(',','').astype('int64')
KOSPI['PSR'] = KOSPI['시가총액'] / KOSPI['매출액']
KOSPI['POR'] = KOSPI['시가총액'] / KOSPI['영업이익']
KOSPI = KOSPI[['종목명','PER','ROE','ROA','PBR','PSR','POR']]
c = ['PER','POR']
KOSPI[c] = KOSPI[c].mask(KOSPI[c] < 0)
KOSPI = KOSPI.dropna(axis = 0)
KOSPI['PBR + PSR'] = KOSPI[['PBR','PSR']].rank(method = 'min').sum(axis = 1)
KOSPI['PBR + PSR + POR'] = KOSPI[['PBR','PSR','POR']].rank(method = 'min').sum(axis = 1)
KOSPI['total_rank'] = (KOSPI[['PER','PBR','PSR']].rank(method = 'min').sum(axis = 1) + KOSPI[['ROE','ROA']].rank(method = 'min',ascending = False).sum(axis = 1))
PBR_PSR = KOSPI[['종목명','PBR','PSR','PBR + PSR']].sort_values(by = 'PBR + PSR')
PBR_PSR = PBR_PSR.drop(['PBR + PSR'],axis = 1)
PBR_PSR.index = list(range(1,len(PBR_PSR)+1))
PBR_PSR

,종목명,PBR,PSR
1,한화생명,0.09,0.056505
2,세아홀딩스,0.14,0.057679
3,동양생명,0.18,0.078174
4,삼천리,0.19,0.085625
5,아세아,0.18,0.107106
...,...,...,...
439,일양약품,7.45,4.744917
440,유나이티드제약,5.26,6.929507
441,엔씨소프트,6.03,10.478897
442,NAVER,6.54,7.748036


In [4]:
PBR_PSR.to_dict('split')

{'index': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,


In [6]:
PBR_PSR_POR = KOSPI[['종목명','PBR','PSR','POR','PBR + PSR + POR']].sort_values(by = 'PBR + PSR + POR')
PBR_PSR_POR = PBR_PSR_POR.drop(['PBR + PSR + POR'],axis = 1)
PBR_PSR_POR.index = list(range(1,len(PBR_PSR_POR)+1))
PBR_PSR_POR

,종목명,PBR,PSR,POR
1,아세아,0.18,0.107365,1.047438
2,넥센,0.21,0.090778,0.924034
3,세아홀딩스,0.14,0.057517,3.168715
4,DB금융투자,0.22,0.164718,1.869565
5,삼천리,0.19,0.086463,3.496495
...,...,...,...,...
439,일진다이아,4.97,5.545624,67.681818
440,한미사이언스,5.91,4.793657,103.013158
441,NAVER,6.50,7.698213,71.479228
442,신풍제약,33.93,40.360042,3828.150000


In [9]:
TOTAL = KOSPI.sort_values(by = 'total_rank')
TOTAL = TOTAL.drop(['PBR + PSR','PBR + PSR + POR','total_rank'], axis = 1)
TOTAL.index = list(range(1,len(TOTAL)+1))

In [7]:
KOSPI = fdr.StockListing('KOSPI')
KOSDAQ = fdr.StockListing('KOSDAQ')
Symbol = KOSDAQ['Symbol']
Name = KOSDAQ['Name']
target = [ i for i in zip(Symbol,Name)]
target = pd.DataFrame(target)
target.columns = ['Symbol','Name']

In [8]:
target

,Symbol,Name
0,060310,3S
1,054620,APS홀딩스
2,265520,AP시스템
3,211270,AP위성
4,035760,CJ ENM
...,...,...
1427,024060,흥구석유
1428,010240,흥국
1429,189980,흥국에프엔비
1430,037440,희림


### 코스닥 상위 종목

In [94]:
ALL = pd.read_excel('all case_v2.xlsx')
c = ['매출액','PER(배)','PBR(배)']
ALL[c] = ALL[c].mask(ALL[c] <= 0)
ALL = ALL.dropna(axis = 0)
ALL.index = list(range(1,len(ALL)+1))
ALL = ALL.drop(['당기순이익','영업이익','EPS(원)','발행주식수(보통주)','total_rank'],axis = 1)
ALL

,종목명,PER(배),PBR(배),ROE(%),ROA(%),매출액,시가총액,PSR,POR
1,이글루시큐리티,8.02,1.24,17.61,11.71,771.0,503.622250,0.653207,3.965530
2,우리넷,5.05,0.62,12.85,9.43,496.0,400.403159,0.807264,6.458115
3,삼영엠텍,8.02,1.24,17.61,11.71,771.0,514.800000,0.667704,4.053543
4,코위버,5.16,0.55,12.01,9.93,600.0,798.439200,1.330732,12.878052
5,시큐브,5.05,0.62,12.85,9.43,496.0,591.600000,1.192742,9.541935
...,...,...,...,...,...,...,...,...,...
201,진성티이씨,236.12,3.70,1.66,0.59,439.0,2371.879274,5.402914,50.465516
202,안랩,257.11,1.16,0.46,0.39,627.0,5377.440135,8.576460,215.097605
203,태광,236.12,3.70,1.66,0.59,439.0,3193.250000,7.273918,67.941489
204,한글과컴퓨터,155.11,10.00,6.66,4.45,124.0,4143.256874,33.413362,517.907109


### 테마별 수익률 (당일 계속 크롤링)

In [131]:
url = 'https://finance.naver.com/sise/sise_group.nhn?type=upjong'
req = requests.get(url)
session = requests.session()
res = session.get(url)

html = req.text
soup = BeautifulSoup(html, 'html.parser')
a = soup.select('div > #contentarea > #contentarea_left > table > tr > td > a')

name = []
for i in range(0,79):
    title = a[i].get_text().strip()
    name.append(title)

b = soup.select('div > #contentarea > #contentarea_left > table > tr > td > span')

value = []
for i in range(0,79):
    title = b[i].get_text().strip()
    value.append(title)
target1 = [ i for i in zip(name,value)]
target1 = pd.DataFrame(target1)
target1.columns = ['테마명','등락률']

target1.to_excel('실시간 테마별 수익.xlsx')

In [132]:
target1.head(10)

,테마명,등락률
0,독립전력생산및에너지거래,+7.02%
1,전기장비,+5.58%
2,조선,+5.41%
3,생명과학도구및서비스,+5.21%
4,증권,+4.86%
5,디스플레이패널,+4.31%
6,에너지장비및서비스,+3.84%
7,기계,+3.07%
8,통신장비,+2.86%
9,건강관리장비와용품,+2.65%


### 실시간 뉴스 타이틀 크롤링

In [133]:
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup

plusUrl = urllib.parse.quote_plus(input('검색어를 입력하세요:'))

pageNum = 1
count = 1

i = input('몇페이지 크롤링 할까요? : ')

lastPage = int(i) * 10 - 9
while pageNum < lastPage + 1:
    url = f'https://search.naver.com/search.naver?date_from=&date_option=0&date_to=&dup_remove=1&nso=&post_blogurl=&post_blogurl_without=&query={plusUrl}&sm=tab_pge&srchby=all&st=sim&where=post&start={pageNum}'

    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    title = soup.find_all(class_='sh_blog_title')

    print(f'-----{count}페이지 결과입니다.-----')
    for i in title:
        print(i.attrs['title'])
        print(i.attrs['href'])
    print()
        
    pageNum += 10
    count += 1

검색어를 입력하세요:삼성전자
몇페이지 크롤링 할까요? : 5
-----1페이지 결과입니다.-----
삼성전자 주식 수익률 340배, 35년 장기 투자 그리고 어이없는 양도세 납부.. 그의 이름 양향자..
https://blog.naver.com/djgkrrl1234?Redirect=Log&logNo=222078395976
삼성전자, 상해종합지수, 신한 레버리지 WTI 원유선물ETN, KODEX WTI 원유선물 확인하기~
https://blog.naver.com/horang_j_h?Redirect=Log&logNo=222082813892
2010년에 삼성전자에 투자했었더라면.
https://blog.naver.com/how2invest?Redirect=Log&logNo=222077219143
9월 7일의 기업분석 Letter - 삼성전자(005930)
https://blog.naver.com/ionia17?Redirect=Log&logNo=222082684896
삼성전자? or 테슬라? 응 아냐 엔비디아야!
https://blog.naver.com/peteryim15?Redirect=Log&logNo=222035385112
삼성전자 주가흐름은 왜 지지부진한가?
https://blog.naver.com/masterwu?Redirect=Log&logNo=222074860326
[추억시리즈] 삼성전자 그리고 중소형주
https://tosoha1.blog.me/221769660469
삼성전자 주가 앞으로의 전망이 궁금하다면?
https://blog.naver.com/poohgirlhs?Redirect=Log&logNo=222039528646
삼성전자를 산게 아니고 외국인은 코스피 선물지수를 샀을 뿐이다
https://thyoon55.blog.me/222045613688
삼성전자 -- 엔비디아 훈풍, 비메모리 보폭 확대
https://blog.naver.com/pokara61?Redirect=Log&logNo=222079115275

-----2페이지 결과입니다.----